In [1]:
using ForwardDiff
using LaTeXStrings
using NLsolve
using Plots
using LinearAlgebra
using PyCall
using JLD
using SparseArrays

#### Le but de ces lignes de code est d'implémenter un schéma aux différences finies pour l'équation d'advection-diffusion en 2D.

On définit les paramètres de la grille de discrétisation en espace : N, num_cells, dx.
On définit également le nombre d'espèces chimiques num_species et le pas de temps dt. 

In [2]:
# N: nombre d'intervalles de discrétisation dans chaque dimension
const N = 200;

# num_cells: nombre de cellules
const num_cells =  N*N;

const dx = 1.0/N;
const dy = 1.0/N
const dt = 0.01;

#T1= 0.25
T = 3

const grid1D = range(0, 1, length=N) |> collect;

getNumCase est la fonction qui renvoie à un couple d'entiers $i,j$ tels que $1\leq i,j \leq N$ renvoie le numéro de la case $1\leq m \leq N^2$ correspondante. 

In [3]:
#Retourne l'indice de la case m en fonction des indices (i,j) (1 <= i,j <= N) 
function getNumCase(i,j)
   return (i-1)*N+j;
end

getNumCase (generic function with 1 method)

getIndicesCase fait l'opération inverse de la fonction getNumCase ci-dessus. 

In [4]:
#Retourne l'indice de la case m en fonction des indices (i,j) ( 1<= i,j <= N) 
function getIndicesCase(m)
   j = m - floor(Int, (m-1)/N)*N
   i = floor(Int, (m-j)/N) +1 
   return i,j
end

getIndicesCase (generic function with 1 method)

Définition de la condition initiale

In [5]:
U0 = zeros(num_cells);

function g(x1,x2)
    return 10*(cos(2*pi*(- 5*(x1-0.2)^2 - 4*(x2-0.7)^2))^2
end

for i=1:N
    for j=1:N
        xi = (i-0.5)*dx
        yj = (j-0.5)*dy
        m = getNumCase(i,j)
        U0[m] = g(xi,yj)
    end
end

LoadError: syntax: missing comma or ) in argument list

Définition de la fonction $b$

In [6]:
function b1(x1,x2)
    return 10*(x2-0.5) 
end

function b2(x1,x2)
    return -10*((x1-0.5))
end

b2 (generic function with 1 method)

Définition de la matrice du Laplacien en 2D 

In [7]:
Lap = spzeros(num_cells, num_cells)
for i=1:N
    for j=1:N
        m = getNumCase(i,j)
        Lap[m,m] = -2.0/(dx*dx) -2.0/(dy*dy)
       
        if (i<N)
            mpx = getNumCase(i+1,j)
        else
            mpx =  getNumCase(1,j)
        end
        Lap[m,mpx] = 1.0/(dx*dx)
       
        if (i>1)     
            mmx = getNumCase(i-1,j)
        else
            mmx =  getNumCase(N,j)
        end
        Lap[m,mmx] = 1.0/(dx*dx)
        
        if (j<N)   
            mpy = getNumCase(i,j+1)
        else
            mpy =  getNumCase(i,1)
        end
        Lap[m,mpy] = 1.0/(dy*dy)
        
   
        if (j>1)
            mmy = getNumCase(i,j-1)
        else
            mmy =  getNumCase(i,N)
        end
        Lap[m,mmy] = 1.0/(dy*dy)
    end
end     

Définition de la matrice $b\cdot \nabla$ 

In [8]:
BGrad = spzeros(num_cells, num_cells)
for i=1:N
    for j=1:N
        xi= (i-0.5)*dx
        yj= (j-0.5)*dy
        m = getNumCase(i,j)
        BGrad[m,m] = 0
       
        if (i<N)
            mpx = getNumCase(i+1,j)
        else
            mpx =  getNumCase(1,j)
        end
        BGrad[m,mpx] = 1.0/(2*dx)*b1(xi,yj)
       
        if (i>1)     
            mmx = getNumCase(i-1,j)
        else
            mmx =  getNumCase(N,j)
        end
        BGrad[m,mmx] = -1.0/(2*dx)*b1(xi,yj)
        
        if (j<N)   
            mpy = getNumCase(i,j+1)
        else
            mpy =  getNumCase(i,1)
        end
        BGrad[m,mpy] = 1.0/(2*dy)*b2(xi,yj)
        
   
        if (j>1)
            mmy = getNumCase(i,j-1)
        else
            mmy =  getNumCase(i,N)
        end
        BGrad[m,mmy] = -1.0/(2*dy)*b2(xi,yj)
    end
end  

Le but des lignes de code ci-dessous est de définir la matrice Identité

In [9]:
Id = sparse(I,num_cells, num_cells);

Schéma numérique: schéma explicite centré

In [10]:
n=1
eta = 0.05

Kmat = eta*Lap + BGrad
println(size(Kmat))
println(size(U0))

Utab = [U0 U0]
println(size(Utab))
Uold0 = U0

while (n*dt < T)
    print("n = ")
    println(n)
    t = n*dt
    V = dt*Kmat*Uold0
    #Explicite: 
    #Unew = Uold0 + dt*Kmat*Uold0
    
    #Implicite:
    Cmat = Id -dt*Kmat
    Unew = Cmat\Uold0
    
    Utab = [Utab Unew]
    Uold0 = Unew
    n = n+1
end

(40000, 40000)
(40000,)
(40000, 2)
n = 1
n = 2
n = 3
n = 4
n = 5
n = 6
n = 7
n = 8
n = 9
n = 10
n = 11
n = 12
n = 13
n = 14
n = 15
n = 16
n = 17
n = 18
n = 19
n = 20
n = 21
n = 22
n = 23
n = 24
n = 25
n = 26
n = 27
n = 28
n = 29
n = 30
n = 31
n = 32
n = 33
n = 34
n = 35
n = 36
n = 37
n = 38
n = 39
n = 40
n = 41
n = 42
n = 43
n = 44
n = 45
n = 46
n = 47
n = 48
n = 49
n = 50
n = 51
n = 52
n = 53
n = 54
n = 55
n = 56
n = 57
n = 58
n = 59
n = 60
n = 61
n = 62
n = 63
n = 64
n = 65
n = 66
n = 67
n = 68
n = 69
n = 70
n = 71
n = 72
n = 73
n = 74
n = 75
n = 76
n = 77
n = 78
n = 79
n = 80
n = 81
n = 82
n = 83
n = 84
n = 85
n = 86
n = 87
n = 88
n = 89
n = 90
n = 91
n = 92
n = 93
n = 94
n = 95
n = 96
n = 97
n = 98
n = 99
n = 100
n = 101
n = 102
n = 103
n = 104
n = 105
n = 106
n = 107
n = 108
n = 109
n = 110
n = 111
n = 112
n = 113
n = 114
n = 115
n = 116
n = 117
n = 118
n = 119
n = 120
n = 121
n = 122
n = 123
n = 124
n = 125
n = 126
n = 127
n = 128
n = 129
n = 130
n = 131
n = 132
n = 133
n = 134
n

In [11]:
anim1 = @animate for p in 1:n
    U = Utab[:,p]
    Umat = reshape(U,(N,N))
    #print(size(Z))
    heatmap(grid1D, grid1D, Umat, title = "Evolution de u(t,x)", xlabel = "x", ylabel = "y")
    #plot(grid1D,grid1D,Z,zlims=(0,1))
end
mp4(anim1, "anim_u_wave_2D.mp4", fps = 1)

GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definition is invalid in routine CELLARRAY
invalid range
GKS: Rectangle definition is invalid in routine SET_WINDOW
GKS: Rectangle definitio

Plots.AnimatedGif("/home/ehrlache/ondes/anim_u_wave_2D.mp4")

In [ ]:
masstab = [0]
for p=2:n
    mass = sum(Utab[:,p])
    masstab = [masstab mass]
end

In [ ]:
plot(masstab')